<a href="https://colab.research.google.com/github/dotRelith/even-faster-stable-diffusion/blob/main/even-faster-stable-diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@markdown # Mount Google Drive

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#@markdown # Check GPU
gpu_info = !nvidia-smi

In [ ]:
#@markdown # Install AUTOMATIC1111-web-ui
from IPython.display import clear_output

!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui /content/gdrive/MyDrive/stable-diffusion-webui
clear_output()
print('[1;32mDONE!')

DONE!


In [2]:
#@markdown # **MODEL DOWNLOAD/LOAD**
from IPython.display import clear_output
import os
#@markdown ---
#@markdown > Direct Google Drive path where you stored all your models(.ckpt files).<br>( Leave empty to use Colab HardDrive )
CKPT_directory_path = '' #@param {type:"string"}
#@markdown ---
#@markdown > Models to download.<br>(Check boxes to download)
AnythingV3 = True #@param {type:"boolean"}
AnimeFullLatest = False #@param {type:"boolean"}
StableDiffusion_v1_5 = False #@param {type:"boolean"}
WaifuDiffusion_v1_3 = False #@param {type:"boolean"}
#@markdown ---
#@markdown > Direct model link<br>(Leave empty to ignore)
FirstCustomModel = '' #@param {type:"string"}
SecondCustomModel = '' #@param {type:"string"}
ThirdCustomModel = '' #@param {type:"string"}
#@markdown ---

if CKPT_directory_path == '':
  CKPT_directory_path = '/content/models'
  if not os.path.exists(CKPT_directory_path):
    os.makedirs(CKPT_directory_path)

%cd "$CKPT_directory_path"

if AnimeFullLatest == True:
  if not os.path.exists("$CKPT_directory_path/novelailatest-pruned.ckpt"):
    !wget https://huggingface.co/a1079602570/animefull-final-pruned/resolve/main/novelailatest-pruned.ckpt
if AnythingV3 == True:
  if not os.path.exists("$CKPT_directory_path/Anything-V3.0-pruned.ckpt"):
    !wget https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/Anything-V3.0-pruned.ckpt
  if not os.path.exists("$CKPT_directory_path/Anything-V3.0.vae.pt"):
    !wget https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/Anything-V3.0.vae.pt
if StableDiffusion_v1_5 == True:
  if not os.path.exists("$CKPT_directory_path/v1-5-pruned.ckpt"):
    !wget https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned.ckpt
if WaifuDiffusion_v1_3 == True:
  if not os.path.exists("$CKPT_directory_path/wd-v1-3-float16.ckpt"):
    !wget https://huggingface.co/hakurei/waifu-diffusion-v1-3/resolve/main/wd-v1-3-float16.ckpt
  
clear_output()
print('[1;32mCOMMON MODELS DONE!')

if FirstCustomModel != '':
  !wget "$FirstCustomModel"
if SecondCustomModel != '':
  !wget "$SecondCustomModel"
if ThirdCustomModel != '':
  !wget "$ThirdCustomModel"  

DONE!


In [ ]:
!pip install https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+189828c.d20221207-cp38-cp38-linux_x86_64.whl
%cd /content/gdrive/MyDrive/stable-diffusion-webui/
!python launch.py --enable-insecure-extension-access --xformers --ckpt-dir "$CKPT_directory_path" --deepdanbooru --precision full --no-half --share --lowram

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 102.9 MB 54 kB/s 
/content/gdrive/.shortcut-targets-by-id/1dX-EdKJc68925M5ZP2Jlg3vZJNFI1a_v/stable-diffusion-webui
Python 3.8.16 (default, Dec  7 2022, 01:12:13) 
[GCC 7.5.0]
Commit hash: 4af3ca5393151d61363c30eef4965e694eeac15e
Installing gfpgan
Installing clip
Installing open_clip
Installing requirements for CodeFormer
Installing requirements for Web UI
Installing requirements for scikit_learn

Launching Web UI with arguments: --enable-insecure-extension-access --xformers --ckpt-dir /content/models --deepdanbooru --precision full --no-half --share --lowram
Checkpoint wd-v1-3-float16.ckpt [84692140] not found; loading fallback Anything-V3.0-pruned.ckpt [2700c435]
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
Downloading: 100% 939k/939k [00:01<00:00, 873kB/s] 
Downloading: 100% 512k/512k [00:01<00:00, 470kB/s]